In [1]:
import random
import threading
import time

In [2]:
# Constants for order types
BUY = "Buy"
SELL = "Sell"

In [3]:
# Order class to represent each order
class Order:
    def __init__(self, order_type, ticker, quantity, price):
        self.order_type = order_type
        self.ticker = ticker
        self.quantity = quantity
        self.price = price
        self.next = None

    def __repr__(self):
        return f"Order({self.order_type}, {self.ticker}, {self.quantity}, {self.price})"

# Node class for linked list implementation
class OrderNode:
    def __init__(self, order):
        self.order = order
        self.next = None

In [4]:
# StockOrderBook for handling Buy and Sell Orders for each ticker
class StockOrderBook:
    def __init__(self):
        self.buy_orders = None
        self.sell_orders = None

    def add_buy_order(self, order):
        # Insert buy orders in descending order of price
        new_node = OrderNode(order)
        if not self.buy_orders or self.buy_orders.order.price < order.price:
            new_node.next = self.buy_orders
            self.buy_orders = new_node
            print(f"Added Buy Order: {order}")
            return

        current = self.buy_orders
        while current.next and current.next.order.price >= order.price:
            current = current.next

        new_node.next = current.next
        current.next = new_node
        print(f"Added Buy Order: {order}")

    def add_sell_order(self, order):
        # Insert sell orders in ascending order of price
        new_node = OrderNode(order)
        if not self.sell_orders or self.sell_orders.order.price > order.price:
            new_node.next = self.sell_orders
            self.sell_orders = new_node
            print(f"Added Sell Order: {order}")
            return

        current = self.sell_orders
        while current.next and current.next.order.price <= order.price:
            current = current.next

        new_node.next = current.next
        current.next = new_node
        print(f"Added Sell Order: {order}")

    def match_orders(self):
        matched_orders = []
        # Matching buy and sell orders
        buy = self.buy_orders
        sell = self.sell_orders

        while buy and sell and buy.order.price >= sell.order.price:
            match_quantity = min(buy.order.quantity, sell.order.quantity)
            matched_orders.append((buy.order, sell.order, match_quantity))

            # Adjust quantities
            buy.order.quantity -= match_quantity
            sell.order.quantity -= match_quantity

            if buy.order.quantity == 0:
                self.buy_orders = buy.next
            if sell.order.quantity == 0:
                self.sell_orders = sell.next

            buy = self.buy_orders
            sell = self.sell_orders

        return matched_orders

In [5]:
# Global dictionary to store order books for each ticker
stock_order_books = {f"Ticker{i}": StockOrderBook() for i in range(1, 1025)}

In [6]:
# StockTradingEngine for adding orders and matching them
class StockTradingEngine:
    def __init__(self):
        self.lock = threading.Lock()

    def add_order(self, order_type, ticker, quantity, price):
        """ Adds order to the order book and attempts to match the orders. """
        with self.lock:
            order = Order(order_type, ticker, quantity, price)
            order_book = stock_order_books[ticker]

            if order_type == BUY:
                order_book.add_buy_order(order)
            elif order_type == SELL:
                order_book.add_sell_order(order)

            self.match_orders(ticker)

    def match_orders(self, ticker):
        """ Match buy and sell orders for a given ticker. """
        order_book = stock_order_books[ticker]
        matched_orders = order_book.match_orders()

        for buy_order, sell_order, quantity in matched_orders:
            print(f"Matched {quantity} units of {buy_order.ticker} at {buy_order.price} (Buy) and {sell_order.price} (Sell)")

# Function to simulate adding orders randomly
def add_order(engine):
    order_type = random.choice([BUY, SELL])
    ticker = f"Ticker{random.randint(1, 1024)}"
    quantity = random.randint(1, 100)
    price = random.uniform(10, 1000)

    engine.add_order(order_type, ticker, quantity, price)

# Wrapper function to simulate random stock orders being placed and matched
def simulate_stock_market(engine):
    while True:
        # Randomly add an order
        add_order(engine)

        # Sleep for a random interval to simulate real-time market
        time.sleep(random.uniform(0.1, 1))


In [7]:
# Start the simulation in a separate thread
if __name__ == "__main__":
    engine = StockTradingEngine()
    threading.Thread(target=simulate_stock_market, args=(engine,), daemon=True).start()

    # Keep the main thread alive to simulate the market
    while True:
        time.sleep(1)

Added Sell Order: Order(Sell, Ticker119, 81, 272.8001942620634)
Added Sell Order: Order(Sell, Ticker717, 34, 604.0297324374169)
Added Buy Order: Order(Buy, Ticker303, 1, 852.5205279881646)
Added Buy Order: Order(Buy, Ticker37, 98, 513.8570016770798)
Added Sell Order: Order(Sell, Ticker948, 62, 478.77936973928433)
Added Buy Order: Order(Buy, Ticker901, 35, 493.434382414578)
Added Sell Order: Order(Sell, Ticker677, 31, 659.5184995192046)
Added Sell Order: Order(Sell, Ticker211, 37, 410.32755090878004)
Added Buy Order: Order(Buy, Ticker514, 20, 664.4177702137406)
Added Sell Order: Order(Sell, Ticker833, 52, 931.9979601165032)
Added Sell Order: Order(Sell, Ticker1011, 62, 742.6777750119109)
Added Buy Order: Order(Buy, Ticker827, 55, 495.369813456694)
Added Sell Order: Order(Sell, Ticker162, 78, 579.329345553979)
Added Buy Order: Order(Buy, Ticker107, 59, 420.55862236215603)
Added Buy Order: Order(Buy, Ticker550, 98, 754.5718043400111)
Added Buy Order: Order(Buy, Ticker692, 41, 126.85237955

KeyboardInterrupt: 